**Introduction:**

In this notebook, we implement a *retrival*-based chatbot, which is trained to provide the best possible response from a database of predefined responses using ML techniques. 

**Required packages:**

In [26]:
import numpy as np
import json

import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Loading the dataset:**

In [11]:
with open('intents.json', 'r') as f:
  intents = json.load(f)

Let's take a look at the dataset:

In [38]:
intents['intents'][0]

{'patterns': ['Hi',
  'Hey',
  'How are you',
  'Is anyone there?',
  'Hello',
  'Good day'],
 'responses': ['Hey :-)',
  'Hello, thanks for visiting',
  'Hi there, what can I do for you?',
  'Hi there, how can I help?'],
 'tag': 'greeting'}

As can be seen, it has several sample question patterns, several possible responses, and also a tag. The tag is used to categorize the input sentence to one of the given categories. 

**Preprocess the data:**

In [39]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
  """
  words is the list of all words in dictionary
  """
    
  # stem each word
  sentence_words = [stem(word) for word in tokenized_sentence]
  # initialize bag with 0 for each word
  bag = np.zeros(len(words), dtype=np.float32)
  for idx, w in enumerate(words):
      if w in sentence_words: 
          bag[idx] = 1

  return bag

Now, we process the given dataset to prepare a dictionary (list of all unique words) and also the list of all available tags.

In [40]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))


# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

**Creating training data:**

In [41]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)



class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
        


**Hyperparameters:**

In [42]:
# Hyper-parameters 
num_epochs = 10000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

In [43]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)


**Creating the neural network:**

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [32]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Training Loop:**

In [33]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')



Epoch [100/10000], Loss: 1.1567
Epoch [200/10000], Loss: 1.3699
Epoch [300/10000], Loss: 0.0470
Epoch [400/10000], Loss: 0.0130
Epoch [500/10000], Loss: 0.0161
Epoch [600/10000], Loss: 0.0020
Epoch [700/10000], Loss: 0.0009
Epoch [800/10000], Loss: 0.0010
Epoch [900/10000], Loss: 0.0022
Epoch [1000/10000], Loss: 0.0003
Epoch [1100/10000], Loss: 0.0001
Epoch [1200/10000], Loss: 0.0003
Epoch [1300/10000], Loss: 0.0008
Epoch [1400/10000], Loss: 0.0001
Epoch [1500/10000], Loss: 0.0001
Epoch [1600/10000], Loss: 0.0001
Epoch [1700/10000], Loss: 0.0000
Epoch [1800/10000], Loss: 0.0001
Epoch [1900/10000], Loss: 0.0000
Epoch [2000/10000], Loss: 0.0000
Epoch [2100/10000], Loss: 0.0000
Epoch [2200/10000], Loss: 0.0000
Epoch [2300/10000], Loss: 0.0000
Epoch [2400/10000], Loss: 0.0000
Epoch [2500/10000], Loss: 0.0000
Epoch [2600/10000], Loss: 0.0000
Epoch [2700/10000], Loss: 0.0000
Epoch [2800/10000], Loss: 0.0000
Epoch [2900/10000], Loss: 0.0000
Epoch [3000/10000], Loss: 0.0000
Epoch [3100/10000],

**Saving the model, parameters, and processed data:**

In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

**Loading the model:**

In [ ]:
FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)

**Testing our traind bot:**

In [48]:

model.eval()

bot_name = "bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.8:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: Hi
bot: Hi there, how can I help?
You: Are you selling coffee?
bot: We have coffee and tea
You: Do you accept AMEX?
bot: We accept VISA, Mastercard and Paypal
You: My last question, how does the delivery take?
bot: Shipping takes 2-4 days
You: perfect. Thank you!
bot: Happy to help!
You: bye
bot: See you later, thanks for visiting
You: quit
